In [1]:
import pandas as pd
from main import Platform, Coin, Transaction

import defi_tools as dft

portfolio = pd.read_csv('portfolio.csv')
portfolio.fillna(0, inplace=True)
portfolio.drop('Unnamed: 0', axis=1, inplace=True)
portfolio.rename(columns={'Unnamed: 1':'id'}, inplace=True)
portfolio.rename(columns={'Coinbase Pro':'Coinbase_Pro'}, inplace=True)
portfolio.set_index('id', inplace=True)


In [2]:
platforms = {k:'' for k in portfolio.columns}
ids = portfolio.index.tolist()
prices = {i:dft.geckoPrice(i, 'usd')[i]['usd'] for i in ids} 
tickers = {i:dft.geckoGetSymbol(i) for i in ids}

In [3]:
for p in platforms.keys():
    platforms[p] = Platform(p)
    for i in ids:
        coin = Coin(coin=i, ticker=tickers[i], price=float(prices[i]), holdings=portfolio.loc[i][p])
        platforms[p].add(coin)

In [4]:
platforms['Coinbase_Pro'].tokens(token='ethereum')

0.35

In [5]:
platforms['Coinbase_Pro'].show()

Total Value: 13380.797286700003


,coin,holdings,value,allocation
0,ETH,3.500000e-01,1123.167500,0.083939
1,USDC,1.000000e+00,1.000000,0.000075
2,BTC,7.000000e-02,2938.320000,0.219592
3,ADA,2.755000e+03,3416.200000,0.255306
4,LINK,9.998000e+01,2600.479800,0.194344
5,DOT,5.200000e+01,1305.200000,0.097543
6,UNI,8.000000e+01,1279.200000,0.095600
7,LRC,3.370000e+02,569.530000,0.042563
8,CRO,1.030000e+00,0.487283,0.000036
9,USDT,6.270400e-02,0.062704,0.000005


In [6]:
cb_pro = platforms['Coinbase_Pro']
cb = platforms['Coinbase']
Transaction.transfer(origin=cb_pro, destination=cb, token='ethereum', amount=0.01, fee=0.0001)
platforms['Coinbase_Pro'].show()

Transferred 0.01 ETH from Coinbase_Pro to Coinbase


,coin,holdings,value,allocation
0,ETH,3.399000e-01,1123.167500,0.083939
1,USDC,1.000000e+00,1.000000,0.000075
2,BTC,7.000000e-02,2938.320000,0.219592
3,ADA,2.755000e+03,3416.200000,0.255306
4,LINK,9.998000e+01,2600.479800,0.194344
5,DOT,5.200000e+01,1305.200000,0.097543
6,UNI,8.000000e+01,1279.200000,0.095600
7,LRC,3.370000e+02,569.530000,0.042563
8,CRO,1.030000e+00,0.487283,0.000036
9,USDT,6.270400e-02,0.062704,0.000005


In [7]:
platforms['Coinbase'].show()

Total Value: 159.25151322000002


,coin,holdings,value,allocation
0,ETH,0.010000,0.000000,0.000000
1,USDC,0.003664,0.003664,0.000023
2,BTC,0.001958,82.177255,0.516022
3,DOGE,265.480000,41.558239,0.260960
4,XLM,70.190000,18.501382,0.116177
5,BAT,10.380000,11.106600,0.069743
6,COMP,0.030063,5.904373,0.037076


In [8]:
cb_pro.add(Coin(coin='USD', ticker='USD', price=1, holdings=2000))

In [9]:
Transaction.transact(cb_pro, 'terra-luna', amount=1000, value=1000, type='buy')

Bought 1000 LUNA.


In [10]:
cb_pro.show()

,coin,holdings,value,allocation
0,ETH,3.399000e-01,1123.167500,0.083939
1,USDC,1.000000e+00,1.000000,0.000075
2,BTC,7.000000e-02,2938.320000,0.219592
3,ADA,2.755000e+03,3416.200000,0.255306
4,LINK,9.998000e+01,2600.479800,0.194344
5,DOT,5.200000e+01,1305.200000,0.097543
6,UNI,8.000000e+01,1279.200000,0.095600
7,LRC,3.370000e+02,569.530000,0.042563
8,CRO,1.030000e+00,0.487283,0.000036
9,LUNA,1.000000e+03,0.000000,0.000000


In [11]:
Transaction.transact(cb_pro, 'terra-luna', amount=10, value=770, type='buy')
Transaction.transact(cb_pro, 'terra-luna', amount=1, value=77, type='sell')

Bought 10 LUNA.
Sold 1 LUNA.


In [ ]:
def geckoPriceAt(token, quote, date):
    """get price of token at historical date
    
    Args:
        token (string): ie "bitcoin"
        quote (fiat or quote currency): ie: "usd"
    
    Returns:
        dictionary: Returns pairs quotes
    """

    url = "https://api.coingecko.com/api/v3/coins/{token}/history?date={date}}&localization=false"
    
    r = requests.get(url).json()